![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

## Project Instructions
The Head Data Scientist at Training Data Ltd. has asked you to create a DataFrame called ds_jobs_transformed that stores the data in customer_train.csv much more efficiently. Specifically, they have set the following requirements:

- Columns containing categories with only two factors must be stored as Booleans (bool).
- Columns containing integers only must be stored as 32-bit integers (int32).
- Columns containing floats must be stored as 16-bit floats (float16).
- Columns containing nominal categorical data must be stored as the category data type.
- Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.
The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.
If you call .info() or .memory_usage() methods on ds_jobs and ds_jobs_transformed after you've preprocessed it, you should notice a substantial decrease in memory usage.

In [69]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [70]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [71]:
ds_jobs_transformed.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [72]:
# perform cleaning
# function converting to bool for columns with only two factors
def convert_to_bool(df):
    assert isinstance(df, pd.DataFrame), "Input should be a pandas DataFrame."
    print("Converting columns with only two factors to bool.")
    two_factor_cats = {
    'relevant_experience': {'No relevant experience': False, 'Has relevant experience': True},
    'job_change': {0.0: False, 1.0: True}
    }
    for column in two_factor_cats.keys():
        df[column] = df[column].map(two_factor_cats[column])
        print(f'{column} converted to boolean.')
    print("Conversion to boolean completed.")
    return df
# function converting float64 to float16
def convert_to_float16(df):
    assert isinstance(df, pd.DataFrame), "Input should be a pandas DataFrame."
    print("Converting columns with float64 to float16.")
    columns = df.columns
    for column in columns:
        if df[column].dtype == "float64":
            df[column] = df[column].astype("float16")
            print(f"{column} converted to float16.")
    print("Conversion to float16 completed.")
    return df

def convert_to_int32(df):
    assert isinstance(df, pd.DataFrame), "Input should be a pandas DataFrame."
    columns = df.columns
    print("Converting columns with int64 to int32.")
    for column in columns:
        if df[column].dtype == "int64":
            df[column] = df[column].astype("int32")
            print(f"{column} converted to int32.")
    print("Conversion to int32 completed.")
    return df
def order_category(df):
    assert isinstance(df, pd.DataFrame), "Input should be a pandas DataFrame"
    print("Converting columns with ordered categories to category")
    ordered_cats = {
    'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
    'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
    }
    for order_col in ordered_cats.keys():
        category = pd.CategoricalDtype(ordered_cats[order_col], ordered = True)
        df[order_col] = df[order_col].astype(category)
        print(f'{order_col} converted to ordered category')
    return df

def convert_obj_cat(df):
    assert isinstance(df, pd.DataFrame), "Input should be a pandas DataFrame"
    # convert the remainig column with data type of object to category 
    for col in df.columns:
        if df[col].dtypes == 'object':
            df[col] = df[col].astype('category')
            print(f'{col} converted to category.')
    return df

def filter_df(df):
    assert isinstance(df, pd.DataFrame), "Input should be a pandas DataFrame."
    print("Filtering students with 10 or more years of experience and companies with at least 1000 employees.")
    # since the two are ordered category, we can apply filtering with inequality operators
    # filter to students with 10 or more years of experience
    df = df[df['experience'] >= '10']
    
    # filter to companies with at least 1000 employees
    df = df[df['company_size'] >= '1000-4999'] # company size is of string, and is given on a range, instead of numerical value
    print("Filtering completed.")
    return df

       
       


In [73]:
ds_jobs_transformed = convert_to_bool(ds_jobs_transformed)
ds_jobs_transformed = convert_to_float16(ds_jobs_transformed)
ds_jobs_transformed = convert_to_int32(ds_jobs_transformed)
ds_jobs_transformed = order_category(ds_jobs_transformed)
ds_jobs_transformed = convert_obj_cat(ds_jobs_transformed)
ds_jobs_transformed = filter_df(ds_jobs_transformed)

Converting columns with only two factors to bool.
relevant_experience converted to boolean.
job_change converted to boolean.
Conversion to boolean completed.
Converting columns with float64 to float16.
city_development_index converted to float16.
Conversion to float16 completed.
Converting columns with int64 to int32.
student_id converted to int32.
training_hours converted to int32.
Conversion to int32 completed.
Converting columns with ordered categories to category
enrolled_university converted to ordered category
education_level converted to ordered category
experience converted to ordered category
company_size converted to ordered category
last_new_job converted to ordered category
city converted to category.
gender converted to category.
major_discipline converted to category.
company_type converted to category.
Filtering students with 10 or more years of experience and companies with at least 1000 employees.
Filtering completed.


In [74]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   bool    
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   bool    
dtypes: bool(2), category(9), flo